* Course: YSC4103 MCS Capstone
* Date created: 2019/02/25
* Name: Linfan XIAO
* Description: Implement the Fokas method as described in "Evolution PDEs and augmented eigenfunctions. Finite interval."



# Importing packages

In [3]:
using SymPy
# using Roots
using Distributions
# using IntervalArithmetic
# using IntervalRootFinding
using ApproxFun

# Global variables

In [2]:
tol = 1e-05

1.0e-5

# Helper functions

## `check_all`

Check whether all elements in a not necessarily homogeneous array satisfy a given condition.

Arguments:
* `array`: a generic array, not necessarily homogeneous.
* `condition`: a boolean function to be applied to each element in the array.

In [ ]:
function check_all(array, condition)
    for x in array
        if !condition(x)
            return false
        end
    end
    return true
end

## `set_tol`

Set an appropriate tolerance for checking whether two numbers are approximately equal.

Arguments:
* `x`: A number.
* `y`: A number.

In [ ]:
function set_tol(x::Number, y::Number)
    return tol * mean([x y]) # tol is global variable
end

## `set_tol_matrix`

Set an appropriate tolerance for checking whether two matrices are approximately equal.

Arguments:
* `A`: A matrix with numeric entries.
* `B`: A matrix with numeric entries.

In [ ]:
function set_tol_matrix(A::Array, B::Array)
    if size(A) != size(B)
        throw(error("Matrix dimensions do not match"))
    end
    # Avoid InexactError() when taking norm()
    A = convert(Array{Complex}, A)
    B = convert(Array{Complex}, B)
    return tol * (norm(A,2) + norm(B,2))
end

## `evaluate`

Evaluate a function on a given value.

Arguments:
* `func`: A function of type Julia `Function`, `SymPy.Sym` (absorbed into `Number`), or `Number`.
* `x`: A value on which the function is to be evaluated on.
* `t`: The free symbol in `func` if `func` is a `SymPy.Sym` object, i.e., a symbolic expression.

In [ ]:
function evaluate(func::Union{Function,Number}, x::Number, t = nothing)
    if isa(func, Function)
        return func(x)
    elseif isa(func, SymPy.Sym) # SymPy.Sym must come before Number because SymPy.Sym will be recognized as Number
        return subs(func, t, x)
    else
        return func
    end
end

## `partition`

Generate two-integer partitions ($0$ included) of a given positive integer.

Arguments:
* `n`: A non-negative integer.

In [8]:
function partition(n::Int)
    if n < 0
        throw("Non-negative n required")
    end
    output = []
    for i = 0:n
        j = n - i
        push!(output, (i,j))
    end
    return output
end

partition (generic function with 1 method)

## `symDeriv`

Construct the symbolic expression for the $k$th derivative of a univariate function.

Arguments:
* `u`: A `SymPy.Sym` object that is a symbolic expression in `t`.
* `t`: The free symbol in `u`.
* `k`: The degree of the desired derivative.

In [11]:
function symDeriv(u::SymPy.Sym, t::SymPy.Sym, k::Int)
    if k < 0
        throw(error("Non-negative degree required"))
    end
    y = u
    for i = 1:k
        newY = diff(y, t)
        y = newY
    end
    return y
end

2-element Array{Any,1}:
 (0, 1)
 (1, 0)

## `add_func`

Function addition given by $(f + g)(x) := f(x) + g(x)$.

Arguments:
* `f`: A `Function` or `Number` (convenient representation of constant function).
* `g`: A `Function` or `Number` (convenient representation of constant function).

In [ ]:
function add_func(f::Union{Number, Function}, g::Union{Number, Function})
    function h(x)
        if isa(f, Number)
            if isa(g, Number)
                return f + g
            else
                return f + g(x)
            end
        elseif isa(f, Function)
            if isa(g, Number)
                return f(x) + g
            else
                return f(x) + g(x)
            end
        end
    end
    return h
end

## `mult_func`

Function multiplication given by $(f \cdot g)(x) := f(x) \cdot g(x)$.

Arguments:
* `f`: A `Function` or `Number` (convenient representation of constant function).
* `g`: A `Function` or `Number` (convenient representation of constant function).

In [ ]:
function mult_func(f::Union{Number, Function}, g::Union{Number, Function})
    function h(x)
        if isa(f, Number)
            if isa(g, Number)
                return f * g
            else
                return f * g(x)
            end
        elseif isa(f, Function)
            if isa(g, Number)
                return f(x) * g
            else
                return f(x) * g(x)
            end
        end
    end
    return h
end

## `evaluate_matrix`

Evaluate a matrix of univariate functions at a given value.

Arguments:
* `matrix`: A matrix of univariate functions, where each entry is of type Julia `Function`, `SymPy.Sym` (absorbed into `Number`), or `Number`.
* `a`: A `Number` at which the matrix of functions will be evaluated.
* `t`: The free symbol in the functions that are the matrix's entries if they are `SymPy.Sym` objects.

In [ ]:
function evaluate_matrix(matrix::Array, a::Number, t = nothing)
    (m, n) = size(matrix)
    matrixA = Array{Number}(m,n)
    for i = 1:m
        for j = 1:n
            matrixA[i,j] = evaluate(matrix[i,j], a, t)
        end
    end
    return matrixA
end

## `get_polynomial`

Construct a polynomial 
$$a_nx^n + a_{n-1}x^{n-1} + \cdots + a_1 x + a_0$$
as a Julia `Function` from an array $[a_n, a_{n-1}, \ldots, a_1, a_0]$.

Arguments:
* `coeffList`: An array of numbers that are the coefficients of $x^n, x^{n-1}, \ldots, x, 1$ in the polynomial.

In [ ]:
function get_polynomial(coeffList::Array)
    polynomial = 0
    n = length(coeffList)-1
    for i in 0:n
        newTerm = t -> coeffList[i+1] * t^(n-i)
        polynomial = add_func(polynomial, newTerm)
    end
    return polynomial
end

## `get_polynomialDeriv`

Get the $k$th derivative of a polynomial with known coefficients.

Arguments:
* `coeffList`: An array of numbers that are the coefficients of $x^n, x^{n-1}, \ldots, x, 1$ in the polynomial.
* `k`: The degree of the desired derivative.

In [ ]:
function get_polynomialDeriv(coeffList::Array, k::Int)
    if k < 0
        throw(error("Only nonnegative degrees are allowed"))
    elseif k == 0
        newCoeffList = coeffList
    else
        for counter = 1:k
            n = length(coeffList)
            newCoeffList = hcat([0],[(n-i)*coeffList[i] for i in 1:(n-1)]')
            coeffList = newCoeffList
        end
    end
    return get_polynomial(newCoeffList)
end

# Structs

## `StructDefinitionError`

A struct definition error type is the class of all errors in struct definitions.

In [ ]:
struct StructDefinitionError <: Exception
    msg::String
end

## `SymLinearDifferentialOperator`

A symbolic linear differential operator of order $n$ is encoded by an $1 \times (n+1)$ array of symbolic expressions with at most one free symbol, an interval $[a,b]$, and that free symbol.

Arguments:
* `symPFunctions`: A array of $n+1$ objects of type `SymPy.Sym` or `Number` $[symP_0, symP_1, \ldots, symP_n]$, corresponding to the symbolic linear differential operator $symL$ given by $symLx = symP_0x^{(n)} + symP_1x^{(n-1)} + \cdots + symP_{n-1}x^{(1)} + symP_n x$.
* `interval`: A real interval $[a,b]$ on which the symbolic differential operator $symL$ is defined.
* `t`: The free symbol in each `symPFunction`.

In [ ]:
struct SymLinearDifferentialOperator
    # Entries in the array should be SymPy.Sym or Number. SymPy.Sym seems to be a subtype of Number, i.e., Array{Union{Number,SymPy.Sym}} returns Array{Number}. But specifying symPFunctions as Array{Number,2} gives a MethodError when the entries are Sympy.Sym objects.
    symPFunctions::Array
    interval::Tuple{Number,Number}
    t::SymPy.Sym
    SymLinearDifferentialOperator(symPFunctions::Array, interval::Tuple{Number,Number}, t::SymPy.Sym) =
    try
        symL = new(symPFunctions, interval, t)
        check_symLinearDifferentialOperator_input(symL)
        return symL
    catch err
        throw(err)
    end
end

function check_symLinearDifferentialOperator_input(symL::SymLinearDifferentialOperator)
    symPFunctions, (a,b), t = symL.symPFunctions, symL.interval, symL.t
    for symPFunc in symPFunctions
        if isa(symPFunc, SymPy.Sym)
            if size(free_symbols(symPFunc)) != (1,) && size(free_symbols(symPFunc)) != (0,)
                throw(StructDefinitionError(:"Only one free symbol is allowed in symP_k"))
            end
        elseif !isa(symPFunc, Number)
            throw(StructDefinitionError(:"symP_k should be SymPy.Sym or Number"))
        end
    end
    return true
end

### `LinearDifferentialOperator`

A linear differential operator of order $n$ is encoded by an $1 \times (n+1)$ array of univariate functions, an interval $[a,b]$, and its symbolic expression.

Arguments:
* `pFunctions`: A array of $n+1$ objects of type `Function` or `Number` $[p_0, p_1, \ldots, p_n]$, corresponding to the linear differential operator $L$ given by $Lx = p_0x^{(n)} + p_1x^{(n-1)} + \cdots + p_{n-1}x^{(1)} + p_n x$.
* `interval`: A real interval $[a,b]$ on which the differential operator $L$ is defined.
* `symL`: The `SymLinearDifferentialOperator` corresponding to $L$.

In [ ]:
# symL is an attribute of L that needs to be input by the user. There are checks to make sure symL is indeed the symbolic version of L.
# Principle: Functionalities of Julia Functions >= Functionalities of SymPy. If p_k has no SymPy representation, the only consequence should be that outputs by functions that take L as arugment has no symbolic expression. E.g., we allow L.pFunctions and L.symL.pFunctions to differ.
struct LinearDifferentialOperator
    pFunctions::Array # Array of julia functions or numbers representing constant functions
    interval::Tuple{Number,Number}
    symL::SymLinearDifferentialOperator
    LinearDifferentialOperator(pFunctions::Array, interval::Tuple{Number,Number}, symL::SymLinearDifferentialOperator) =
    try
        L = new(pFunctions, interval, symL)
        check_linearDifferentialOperator_input(L)
        return L
    catch err
        throw(err)
    end
end

# Assume symFunc has only one free symbol, as required by the definition of SymLinearDifferentialOperator. 
# That is, assume the input symFunc comes from SymLinearDifferentialOperator.
function check_func_sym_equal(func::Union{Function,Number}, symFunc, interval::Tuple{Number,Number}, t::SymPy.Sym) # symFunc should be Union{SymPy.Sym, Number}, but somehow SymPy.Sym gets ignored
    (a,b) = interval
    # Randomly sample 1000 points from (a,b) and check if func and symFunc agree on them
    for i = 1:1000
        # Check endpoints
        if i == 1
            x = a
        elseif i == 2
            x = b
        else
            x = rand(Uniform(a,b), 1)[1,1]
        end
        funcEvalX = evaluate(func, x)
        if isa(symFunc, SymPy.Sym)
            symFuncEvalX = SymPy.N(subs(symFunc,t,x))
            # N() converts SymPy.Sym to Number
            # https://docs.sympy.org/latest/modules/evalf.html
            # subs() works no matter symFunc is Number or SymPy.Sym
        else
            symFuncEvalX = symFunc
        end
        tol = set_tol(funcEvalX, symFuncEvalX)
        if !isapprox(real(funcEvalX), real(symFuncEvalX); atol = real(tol)) ||
            !isapprox(imag(funcEvalX), imag(symFuncEvalX); atol = imag(tol))
            println("x = $x")
            println("symFunc = $symFunc")
            println("funcEvalX = $funcEvalX")
            println("symFuncEvalX = $symFuncEvalX")
            return false
        end
    end
    return true
end

# Check whether the inputs of L are valid.
function check_linearDifferentialOperator_input(L::LinearDifferentialOperator)
    pFunctions, (a,b), symL = L.pFunctions, L.interval, L.symL
    symPFunctions, t = symL.symPFunctions, symL.t
    # domainC = Complex(a..b, 0..0) # Domain [a,b] represented in the complex plane
    p0 = pFunctions[1]
    # p0Chebyshev = Fun(p0, a..b) # Chebysev polynomial approximation of p0 on [a,b]
    if !check_all(pFunctions, pFunc -> (isa(pFunc, Function) || isa(pFunc, Number)))
        throw(StructDefinitionError(:"p_k should be Function or Number"))
    elseif length(pFunctions) != length(symPFunctions)
        throw(StructDefinitionError(:"Number of p_k and symP_k do not match"))
    elseif (a,b) != symL.interval
        throw(StructDefinitionError(:"Intervals of L and symL do not match"))
    # # Assume p_k are in C^{n-k}. Check whether p0 vanishes on [a,b]. 
    # # roots() in IntervalRootFinding doesn't work if p0 is sth like t*im - 2*im. Neither does find_zero() in Roots.
    # # ApproxFun.roots() 
    # elseif (isa(p0, Function) && (!isempty(roots(p0Chebyshev)) || all(x->x>b, roots(p0Chebyshev)) || all(x->x<b, roots(p0Chebyshev)) || p0(a) == 0 || p0(b) == 0)) || p0 == 0 
    #     throw(StructDefinitionError(:"p0 vanishes on [a,b]"))
    elseif !all(i -> check_func_sym_equal(pFunctions[i], symPFunctions[i], (a,b), t), 1:length(pFunctions))
        # throw(StructDefinitionError(:"symP_k does not agree with p_k on [a,b]"))
        warn("symP_k does not agree with p_k on [a,b]") # Make this a warning instead of an error because the functionalities of Julia Functions may be more than those of SymPy objects; we do not want to compromise the functionalities of LinearDifferentialOperator because of the restrictions on SymPy.
    else
        return true
    end
end

### `VectorBoundaryForm`

A homogeneous boundary condition
$$Ux = \begin{bmatrix}U_1\\\vdots\\ U_m\end{bmatrix}x = \begin{bmatrix}\sum_{j=1}^n M_{1j}x^{(j-1)}(a) + N_{1j}x^{(j-1)}(b)\\\vdots\\ \sum_{j=1}^n M_{mj}x^{(j-1)}(a) + N_{mj}x^{(j-1)}(b)\end{bmatrix} = \begin{bmatrix}0\\\vdots\\ 0\end{bmatrix}$$
is encoded by an ordered pair of two $m\times n$ matrices $(M, N)$ where
$$M = \begin{bmatrix}M_{11} & \cdots & M_{1n}\\ \vdots & \ddots & \vdots\\ M_{m1} & \cdots & M_{mn}\end{bmatrix},\quad N = \begin{bmatrix}N_{11} & \cdots & N_{1n}\\ \vdots & \ddots & \vdots\\ N_{m1} & \cdots & N_{mn}\end{bmatrix}.$$

Arguments:
* `M`: A matrix whose entries are of type `Number`.
* `N`: A matrix whose entries are of type `Number` and has the same dimension as `N`.

In [ ]:
struct VectorBoundaryForm
    M::Array # Why can't I specify Array{Number,2} without having a MethodError?
    N::Array
    VectorBoundaryForm(M::Array, N::Array) =
    try
        U = new(M, N)
        check_vectorBoundaryForm_input(U)
        return U
    catch err
        throw(err)
    end
end

# Check whether the input matrices that characterize U are valid
function check_vectorBoundaryForm_input(U::VectorBoundaryForm)
    # M, N = U.M, U.N
    # Avoid Inexact() error when taking rank()
    M = convert(Array{Complex}, U.M)
    N = convert(Array{Complex}, U.N)
    if !(check_all(U.M, x -> isa(x, Number)) && check_all(U.N, x -> isa(x, Number)))
        throw(StructDefinitionError(:"Entries of M, N should be Number"))
    elseif size(U.M) != size(U.N)
        throw(StructDefinitionError(:"M, N dimensions do not match"))
    elseif size(U.M)[1] != size(U.M)[2]
        throw(StructDefinitionError(:"M, N should be square matrices"))
    elseif rank(hcat(M, N)) != size(M)[1] # rank() throws weird "InexactError()" when taking some complex matrices
        throw(StructDefinitionError(:"Boundary operators not linearly independent"))
    else
        return true
    end
end

# Construct adjoint boundary conditions

Algorithm to construct a valid adjoint boundary condition from a given (homogeneous) boundary condition based on Chapter 11 in Theory of Ordinary Differential Equations (Coddington & Levinson). The implementation uses Julia functions as main objects but supports symbolic expressions.

## `get_L`

Construct $L$ from $symL$ by converting `symPFunctions` to Julia `Function` objects.

In [ ]:
function get_L(symL::SymLinearDifferentialOperator)
    symPFunctions, (a,b), t = symL.symPFunctions, symL.interval, symL.t
    pFunctions = Array{Union{Function, Number}}(1,length(symPFunctions))
    for i =1:length(symPFunctions)
        symPFunc = symPFunctions[i]
        if isa(symPFunc, SymPy.Sym) # if symPFunc is a SymPy.Sym object
            if t in free_symbols(symPFunc)
                pFunc = lambdify(symPFunc)
            else
                pFunc = N(symPFunc)
            end
        else # symPFunc could be a Number object
            pFunc = symPFunc
        end
        pFunctions[i] = pFunc
    end
    L = LinearDifferentialOperator(pFunctions, (a,b), symL)
    return L
end

# Approximate roots of exponential polynomial

# The Fokas Transform pair